<a href="https://colab.research.google.com/github/Kiran45181/Pyspark/blob/main/pyspark_sales_analysis_full_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Aggregation Problems

In [2]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Create SparkSession
spark = SparkSession.builder.appName("SalesAnalysis").getOrCreate()

# ==========================
# Create DataFrames
# ==========================

# Sales Dataset
sales_data = [
    (1001, 101, 501, "2025-07-10 08:23:00", 3, 25.5),
    (1002, 102, 502, "2025-07-11 09:45:00", 2, 15.0),
    (1003, 103, 503, "2025-07-12 10:15:00", 1, 30.0),
    (1004, 101, 504, "2025-07-13 12:20:00", 5, 25.5),
    (1005, 105, 505, "2025-07-14 14:35:00", 10, 45.0),
    (1006, 102, 506, "2025-07-15 16:00:00", 4, 15.0)
]
sales_columns = ["sale_id", "product_id", "customer_id", "sale_date", "quantity", "price"]
sales_df = spark.createDataFrame(sales_data, sales_columns)

sales_df.show()

+-------+----------+-----------+-------------------+--------+-----+
|sale_id|product_id|customer_id|          sale_date|quantity|price|
+-------+----------+-----------+-------------------+--------+-----+
|   1001|       101|        501|2025-07-10 08:23:00|       3| 25.5|
|   1002|       102|        502|2025-07-11 09:45:00|       2| 15.0|
|   1003|       103|        503|2025-07-12 10:15:00|       1| 30.0|
|   1004|       101|        504|2025-07-13 12:20:00|       5| 25.5|
|   1005|       105|        505|2025-07-14 14:35:00|      10| 45.0|
|   1006|       102|        506|2025-07-15 16:00:00|       4| 15.0|
+-------+----------+-----------+-------------------+--------+-----+



In [3]:
# Product Dataset
product_data = [
    (101, "Widget A", "Gadgets"),
    (102, "Widget B", "Gadgets"),
    (103, "Widget C", "Electronics"),
    (104, "Widget D", "Electronics"),
    (105, "Widget E", "Home & Living")
]
product_columns = ["product_id", "product_name", "category"]
product_df = spark.createDataFrame(product_data, product_columns)

product_df.show()

+----------+------------+-------------+
|product_id|product_name|     category|
+----------+------------+-------------+
|       101|    Widget A|      Gadgets|
|       102|    Widget B|      Gadgets|
|       103|    Widget C|  Electronics|
|       104|    Widget D|  Electronics|
|       105|    Widget E|Home & Living|
+----------+------------+-------------+



In [4]:
# Customer Dataset
customer_data = [
    (501, "Alice", "alice@example.com", "2025-05-20 10:10:00"),
    (502, "Bob", "bob@example.com", "2025-06-15 14:00:00"),
    (503, "Charlie", "charlie@example.com", "2025-04-05 09:50:00"),
    (504, "David", "david@example.com", "2025-07-01 12:25:00"),
    (505, "Emma", "emma@example.com", "2025-07-10 15:30:00"),
    (506, "Frank", "frank@example.com", "2025-03-23 17:00:00")
]
customer_columns = ["customer_id", "customer_name", "email", "join_date"]
customer_df = spark.createDataFrame(customer_data, customer_columns)
customer_df.show()

+-----------+-------------+-------------------+-------------------+
|customer_id|customer_name|              email|          join_date|
+-----------+-------------+-------------------+-------------------+
|        501|        Alice|  alice@example.com|2025-05-20 10:10:00|
|        502|          Bob|    bob@example.com|2025-06-15 14:00:00|
|        503|      Charlie|charlie@example.com|2025-04-05 09:50:00|
|        504|        David|  david@example.com|2025-07-01 12:25:00|
|        505|         Emma|   emma@example.com|2025-07-10 15:30:00|
|        506|        Frank|  frank@example.com|2025-03-23 17:00:00|
+-----------+-------------+-------------------+-------------------+



In [5]:
# 1. Total Revenue Per Product
sales_df = sales_df.withColumn("revenue", F.col("quantity") * F.col("price"))
total_revenue_per_product = sales_df.groupBy("product_id").agg(F.sum("revenue").alias("total_revenue"))
total_revenue_per_product.show()

+----------+-------------+
|product_id|total_revenue|
+----------+-------------+
|       103|         30.0|
|       101|        204.0|
|       102|         90.0|
|       105|        450.0|
+----------+-------------+



In [6]:
# 2. Total Quantity Sold Per Customer
total_quantity_per_customer = sales_df.groupBy("customer_id").agg(F.sum("quantity").alias("total_quantity"))
total_quantity_per_customer.show()

+-----------+--------------+
|customer_id|total_quantity|
+-----------+--------------+
|        502|             2|
|        501|             3|
|        503|             1|
|        504|             5|
|        506|             4|
|        505|            10|
+-----------+--------------+



In [7]:
# 3. Average Revenue Per Customer
avg_revenue_per_customer = sales_df.groupBy("customer_id").agg(F.avg("revenue").alias("avg_revenue"))
avg_revenue_per_customer.show()

+-----------+-----------+
|customer_id|avg_revenue|
+-----------+-----------+
|        502|       30.0|
|        501|       76.5|
|        503|       30.0|
|        504|      127.5|
|        506|       60.0|
|        505|      450.0|
+-----------+-----------+



In [8]:
# 4. Monthly Sales Total
monthly_sales_total = sales_df.withColumn("month", F.month("sale_date")).groupBy("month").agg(F.sum("revenue").alias("total_sales"))
monthly_sales_total.show()

+-----+-----------+
|month|total_sales|
+-----+-----------+
|    7|      774.0|
+-----+-----------+



In [9]:
# 5. Count of Sales Per Category
sales_with_category = sales_df.join(product_df, on="product_id", how="inner")
count_sales_per_category = sales_with_category.groupBy("category").agg(F.count("*").alias("sales_count"))
count_sales_per_category.show()

+-------------+-----------+
|     category|sales_count|
+-------------+-----------+
|  Electronics|          1|
|      Gadgets|          4|
|Home & Living|          1|
+-------------+-----------+



In [10]:
# 6. Top 3 Most Expensive Products Sold
top3_expensive_products = sales_df.groupBy("product_id").agg(F.sum("revenue").alias("total_revenue")).orderBy(F.desc("total_revenue")).limit(3)
top3_expensive_products.show()

+----------+-------------+
|product_id|total_revenue|
+----------+-------------+
|       105|        450.0|
|       101|        204.0|
|       102|         90.0|
+----------+-------------+



## Joins

In [11]:
# 1. Join Sales with Product Information
sales_product_join = sales_df.join(product_df, on="product_id", how="inner")
sales_product_join.show()


+----------+-------+-----------+-------------------+--------+-----+-------+------------+-------------+
|product_id|sale_id|customer_id|          sale_date|quantity|price|revenue|product_name|     category|
+----------+-------+-----------+-------------------+--------+-----+-------+------------+-------------+
|       101|   1001|        501|2025-07-10 08:23:00|       3| 25.5|   76.5|    Widget A|      Gadgets|
|       101|   1004|        504|2025-07-13 12:20:00|       5| 25.5|  127.5|    Widget A|      Gadgets|
|       102|   1002|        502|2025-07-11 09:45:00|       2| 15.0|   30.0|    Widget B|      Gadgets|
|       102|   1006|        506|2025-07-15 16:00:00|       4| 15.0|   60.0|    Widget B|      Gadgets|
|       103|   1003|        503|2025-07-12 10:15:00|       1| 30.0|   30.0|    Widget C|  Electronics|
|       105|   1005|        505|2025-07-14 14:35:00|      10| 45.0|  450.0|    Widget E|Home & Living|
+----------+-------+-----------+-------------------+--------+-----+------

In [12]:
# 2. Join Sales with Customer Information
sales_customer_join = sales_df.join(customer_df, on="customer_id", how="inner")
sales_customer_join.show()

+-----------+-------+----------+-------------------+--------+-----+-------+-------------+-------------------+-------------------+
|customer_id|sale_id|product_id|          sale_date|quantity|price|revenue|customer_name|              email|          join_date|
+-----------+-------+----------+-------------------+--------+-----+-------+-------------+-------------------+-------------------+
|        501|   1001|       101|2025-07-10 08:23:00|       3| 25.5|   76.5|        Alice|  alice@example.com|2025-05-20 10:10:00|
|        502|   1002|       102|2025-07-11 09:45:00|       2| 15.0|   30.0|          Bob|    bob@example.com|2025-06-15 14:00:00|
|        503|   1003|       103|2025-07-12 10:15:00|       1| 30.0|   30.0|      Charlie|charlie@example.com|2025-04-05 09:50:00|
|        504|   1004|       101|2025-07-13 12:20:00|       5| 25.5|  127.5|        David|  david@example.com|2025-07-01 12:25:00|
|        505|   1005|       105|2025-07-14 14:35:00|      10| 45.0|  450.0|         Emma| 

In [13]:
# 3. Inner Join Between Sales and Products (Gadgets Only)
gadgets_sales = sales_df.join(product_df, on="product_id", how="inner").filter(F.col("category") == "Gadgets")
gadgets_sales.show()

+----------+-------+-----------+-------------------+--------+-----+-------+------------+--------+
|product_id|sale_id|customer_id|          sale_date|quantity|price|revenue|product_name|category|
+----------+-------+-----------+-------------------+--------+-----+-------+------------+--------+
|       101|   1001|        501|2025-07-10 08:23:00|       3| 25.5|   76.5|    Widget A| Gadgets|
|       101|   1004|        504|2025-07-13 12:20:00|       5| 25.5|  127.5|    Widget A| Gadgets|
|       102|   1002|        502|2025-07-11 09:45:00|       2| 15.0|   30.0|    Widget B| Gadgets|
|       102|   1006|        506|2025-07-15 16:00:00|       4| 15.0|   60.0|    Widget B| Gadgets|
+----------+-------+-----------+-------------------+--------+-----+-------+------------+--------+



In [14]:
# 4. Left Join Between Sales and Products
left_join_sales_products = sales_df.join(product_df, on="product_id", how="left")
left_join_sales_products.show()

+----------+-------+-----------+-------------------+--------+-----+-------+------------+-------------+
|product_id|sale_id|customer_id|          sale_date|quantity|price|revenue|product_name|     category|
+----------+-------+-----------+-------------------+--------+-----+-------+------------+-------------+
|       103|   1003|        503|2025-07-12 10:15:00|       1| 30.0|   30.0|    Widget C|  Electronics|
|       101|   1001|        501|2025-07-10 08:23:00|       3| 25.5|   76.5|    Widget A|      Gadgets|
|       102|   1002|        502|2025-07-11 09:45:00|       2| 15.0|   30.0|    Widget B|      Gadgets|
|       105|   1005|        505|2025-07-14 14:35:00|      10| 45.0|  450.0|    Widget E|Home & Living|
|       101|   1004|        504|2025-07-13 12:20:00|       5| 25.5|  127.5|    Widget A|      Gadgets|
|       102|   1006|        506|2025-07-15 16:00:00|       4| 15.0|   60.0|    Widget B|      Gadgets|
+----------+-------+-----------+-------------------+--------+-----+------

In [15]:
# 5. Self-Join Sales Table
sales_self_join = sales_df.alias("a").join(
    sales_df.alias("b"),
    (F.col("a.product_id") == F.col("b.product_id")) & (F.col("a.sale_id") != F.col("b.sale_id")),
    how="inner"
)
sales_self_join.show()

+-------+----------+-----------+-------------------+--------+-----+-------+-------+----------+-----------+-------------------+--------+-----+-------+
|sale_id|product_id|customer_id|          sale_date|quantity|price|revenue|sale_id|product_id|customer_id|          sale_date|quantity|price|revenue|
+-------+----------+-----------+-------------------+--------+-----+-------+-------+----------+-----------+-------------------+--------+-----+-------+
|   1001|       101|        501|2025-07-10 08:23:00|       3| 25.5|   76.5|   1004|       101|        504|2025-07-13 12:20:00|       5| 25.5|  127.5|
|   1004|       101|        504|2025-07-13 12:20:00|       5| 25.5|  127.5|   1001|       101|        501|2025-07-10 08:23:00|       3| 25.5|   76.5|
|   1002|       102|        502|2025-07-11 09:45:00|       2| 15.0|   30.0|   1006|       102|        506|2025-07-15 16:00:00|       4| 15.0|   60.0|
|   1006|       102|        506|2025-07-15 16:00:00|       4| 15.0|   60.0|   1002|       102|      

In [16]:
# 6. Full Outer Join Sales and Products
full_outer_join = sales_df.join(product_df, on="product_id", how="outer")
full_outer_join.show()

+----------+-------+-----------+-------------------+--------+-----+-------+------------+-------------+
|product_id|sale_id|customer_id|          sale_date|quantity|price|revenue|product_name|     category|
+----------+-------+-----------+-------------------+--------+-----+-------+------------+-------------+
|       101|   1001|        501|2025-07-10 08:23:00|       3| 25.5|   76.5|    Widget A|      Gadgets|
|       101|   1004|        504|2025-07-13 12:20:00|       5| 25.5|  127.5|    Widget A|      Gadgets|
|       102|   1002|        502|2025-07-11 09:45:00|       2| 15.0|   30.0|    Widget B|      Gadgets|
|       102|   1006|        506|2025-07-15 16:00:00|       4| 15.0|   60.0|    Widget B|      Gadgets|
|       103|   1003|        503|2025-07-12 10:15:00|       1| 30.0|   30.0|    Widget C|  Electronics|
|       104|   NULL|       NULL|               NULL|    NULL| NULL|   NULL|    Widget D|  Electronics|
|       105|   1005|        505|2025-07-14 14:35:00|      10| 45.0|  450.

In [17]:
# 7. Join Multiple Tables (Sales + Product + Customer)
sales_full_details = sales_df.join(product_df, on="product_id", how="inner").join(customer_df, on="customer_id", how="inner")
sales_full_details.show()

+-----------+----------+-------+-------------------+--------+-----+-------+------------+-------------+-------------+-------------------+-------------------+
|customer_id|product_id|sale_id|          sale_date|quantity|price|revenue|product_name|     category|customer_name|              email|          join_date|
+-----------+----------+-------+-------------------+--------+-----+-------+------------+-------------+-------------+-------------------+-------------------+
|        502|       102|   1002|2025-07-11 09:45:00|       2| 15.0|   30.0|    Widget B|      Gadgets|          Bob|    bob@example.com|2025-06-15 14:00:00|
|        501|       101|   1001|2025-07-10 08:23:00|       3| 25.5|   76.5|    Widget A|      Gadgets|        Alice|  alice@example.com|2025-05-20 10:10:00|
|        503|       103|   1003|2025-07-12 10:15:00|       1| 30.0|   30.0|    Widget C|  Electronics|      Charlie|charlie@example.com|2025-04-05 09:50:00|
|        504|       101|   1004|2025-07-13 12:20:00|      

## Additional Transformations

In [18]:
# 1. Filter Data for Specific Date Range
filtered_sales = sales_df.filter(
    (F.col("sale_date") >= "2025-07-10") & (F.col("sale_date") <= "2025-07-15")
)
filtered_sales.show()

+-------+----------+-----------+-------------------+--------+-----+-------+
|sale_id|product_id|customer_id|          sale_date|quantity|price|revenue|
+-------+----------+-----------+-------------------+--------+-----+-------+
|   1001|       101|        501|2025-07-10 08:23:00|       3| 25.5|   76.5|
|   1002|       102|        502|2025-07-11 09:45:00|       2| 15.0|   30.0|
|   1003|       103|        503|2025-07-12 10:15:00|       1| 30.0|   30.0|
|   1004|       101|        504|2025-07-13 12:20:00|       5| 25.5|  127.5|
|   1005|       105|        505|2025-07-14 14:35:00|      10| 45.0|  450.0|
+-------+----------+-----------+-------------------+--------+-----+-------+



In [19]:
# 2. Top 5 Customers by Total Spend
top5_customers = sales_df.groupBy("customer_id") \
    .agg(F.sum("revenue").alias("total_spend")) \
    .orderBy(F.desc("total_spend")) \
    .limit(5)
top5_customers.show()

+-----------+-----------+
|customer_id|total_spend|
+-----------+-----------+
|        505|      450.0|
|        504|      127.5|
|        501|       76.5|
|        506|       60.0|
|        502|       30.0|
+-----------+-----------+



In [20]:
# 3. Categorizing Customers Based on Spend
spend_per_customer = sales_df.groupBy("customer_id") \
    .agg(F.sum("revenue").alias("total_spend")) \
    .withColumn("spend_category",
        F.when(F.col("total_spend") < 100, "Low")
         .when(F.col("total_spend") < 200, "Medium")
         .otherwise("High")
    )
spend_per_customer.show()

+-----------+-----------+--------------+
|customer_id|total_spend|spend_category|
+-----------+-----------+--------------+
|        502|       30.0|           Low|
|        501|       76.5|           Low|
|        503|       30.0|           Low|
|        504|      127.5|        Medium|
|        506|       60.0|           Low|
|        505|      450.0|          High|
+-----------+-----------+--------------+



In [21]:
# 4. Finding First and Last Purchase for Each Customer
purchase_dates = sales_df.groupBy("customer_id") \
    .agg(
        F.min("sale_date").alias("first_purchase"),
        F.max("sale_date").alias("last_purchase")
    )
purchase_dates.show()

+-----------+-------------------+-------------------+
|customer_id|     first_purchase|      last_purchase|
+-----------+-------------------+-------------------+
|        501|2025-07-10 08:23:00|2025-07-10 08:23:00|
|        502|2025-07-11 09:45:00|2025-07-11 09:45:00|
|        503|2025-07-12 10:15:00|2025-07-12 10:15:00|
|        504|2025-07-13 12:20:00|2025-07-13 12:20:00|
|        505|2025-07-14 14:35:00|2025-07-14 14:35:00|
|        506|2025-07-15 16:00:00|2025-07-15 16:00:00|
+-----------+-------------------+-------------------+



In [22]:
# 5. Customer Churn (No Purchase in Last 30 Days)
last_purchase = sales_df.groupBy("customer_id") \
    .agg(F.max("sale_date").alias("last_purchase"))
churn_customers = last_purchase.filter(
    F.datediff(F.current_date(), F.col("last_purchase")) > 30
)
churn_customers.show()

+-----------+-------------+
|customer_id|last_purchase|
+-----------+-------------+
+-----------+-------------+

